![Illustration of silhouetted heads](mentalhealth.jpg)

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties than the general population, and that social connectedness (belonging to a social group) and acculturative stress (stress associated with joining a new culture) are predictive of depression.


Explore the `students` data using PostgreSQL to find out if you would come to a similar conclusion for international students and see if the length of stay is a contributing factor.

Here is a data description of the columns you may find helpful.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

In [16]:
-- the data in students
SELECT *
FROM students;

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24.0,4.0,5.0,Long,3.0,Average,5.0,High,,Yes,No,No,No,0.0,Min,34.0,23.0,9.0,11.0,8.0,11.0,2.0,27.0,91.0,5.0,5.0,6.0,3.0,2.0,1.0,4.0,1.0,3.0,4.0,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28.0,5.0,1.0,Short,4.0,High,4.0,High,,No,No,No,No,2.0,Min,48.0,8.0,7.0,5.0,4.0,3.0,2.0,10.0,39.0,7.0,7.0,7.0,4.0,4.0,4.0,4.0,1.0,1.0,1.0,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25.0,4.0,6.0,Long,4.0,High,4.0,High,Yes,Yes,No,No,No,2.0,Min,41.0,13.0,4.0,7.0,6.0,4.0,3.0,14.0,51.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29.0,5.0,1.0,Short,2.0,Low,3.0,Average,No,No,No,No,No,3.0,Min,37.0,16.0,10.0,10.0,8.0,6.0,4.0,21.0,75.0,5.0,5.0,5.0,5.0,5.0,2.0,2.0,2.0,4.0,4.0,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28.0,5.0,1.0,Short,1.0,Low,3.0,Average,Yes,No,No,No,No,3.0,Min,37.0,15.0,12.0,5.0,8.0,7.0,4.0,31.0,82.0,5.0,5.0,5.0,2.0,5.0,2.0,5.0,5.0,4.0,4.0,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,140,,,,,,,,,
282,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137,131,,,,,,,,,
283,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,202,,,,,,,,,
284,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,207,,,,,,,,,


### PROJECT 
### _"Do International Students Face Higher Mental Health Risks? An Exploratory Analysis Using SQL"_

## Project Objectives

### - Compare international vs. domestic students on depression levels.
### - Analyze whether length of stay reduces or increases depression risk.
### - Investigate the role of social connectedness in protecting against depression.
### - Examine how acculturative stress contributes to depression.   
### - Explore demographic differences (academic level, age, language proficiency).

## 0) Quick peek & schema

In [3]:
-- 1) quick row-count and sample
SELECT COUNT(*) AS total_rows FROM students;
SELECT * FROM students LIMIT 20;

-- 2) columns & types
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'students'
ORDER BY ordinal_position;


,column_name,data_type
0,inter_dom,text
1,region,text
2,gender,text
3,academic,text
4,age,integer
5,age_cate,integer
6,stay,integer
7,stay_cate,text
8,japanese,integer
9,japanese_cate,text


## 1) Count missing / bad values per column

### (Shows how "complete" the dataset is)

In [4]:
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN inter_dom IS NULL OR TRIM(inter_dom) = '' THEN 1 ELSE 0 END) AS inter_dom_nulls,
  SUM(CASE WHEN japanese_cate IS NULL OR TRIM(japanese_cate) = '' THEN 1 ELSE 0 END) AS japanese_nulls,
  SUM(CASE WHEN english_cate IS NULL OR TRIM(english_cate) = '' THEN 1 ELSE 0 END) AS english_nulls,
  SUM(CASE WHEN academic IS NULL OR TRIM(academic) = '' THEN 1 ELSE 0 END) AS academic_nulls,
  SUM(CASE WHEN age IS NULL OR TRIM(age::text) = '' THEN 1 ELSE 0 END) AS age_nulls,
  SUM(CASE WHEN stay IS NULL OR TRIM(stay::text) = '' THEN 1 ELSE 0 END) AS stay_nulls,
  SUM(CASE WHEN todep IS NULL THEN 1 ELSE 0 END) AS todep_nulls,
  SUM(CASE WHEN tosc IS NULL THEN 1 ELSE 0 END) AS tosc_nulls,
  SUM(CASE WHEN toas IS NULL THEN 1 ELSE 0 END) AS toas_nulls
FROM students;


,total_rows,inter_dom_nulls,japanese_nulls,english_nulls,academic_nulls,age_nulls,stay_nulls,todep_nulls,tosc_nulls,toas_nulls
0,286,18,18,18,18,18,18,18,18,18


## 2) Standardize / normalize text categories

In [7]:
SELECT DISTINCT TRIM(LOWER(inter_dom)) AS raw_inter_dom
FROM students
ORDER BY 1;


,raw_inter_dom
0,
1,dom
2,inter


### Clean categories using a CTE

In [8]:
WITH cleaned AS (
  SELECT
    *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international', 'intl', 'i', 'foreign')
        THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic', 'local', 'japanese', 'd')
        THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std
  FROM students
)
SELECT inter_dom, inter_dom_std
FROM cleaned
LIMIT 20;


,inter_dom,inter_dom_std
0,Inter,NaN
1,Inter,NaN
2,Inter,NaN
3,Inter,NaN
4,Inter,NaN
5,Inter,NaN
6,Inter,NaN
7,Inter,NaN
8,Inter,NaN
9,Inter,NaN


### Drop NULLs before analysis

In [9]:
WITH cleaned AS (
  SELECT
    *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international', 'intl', 'i', 'foreign')
        THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic', 'local', 'japanese', 'd')
        THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std
  FROM students
)
SELECT inter_dom_std,
       COUNT(*) AS num_students,
       AVG(todep) AS avg_depression,
       AVG(tosc) AS avg_connectedness,
       AVG(toas) AS avg_stress
FROM cleaned
WHERE inter_dom_std IS NOT NULL
  AND todep IS NOT NULL
GROUP BY inter_dom_std;


,inter_dom_std,num_students,avg_depression,avg_connectedness,avg_stress


### Check missingness & inspect raw category values

In [11]:
-- A) Missingness counts
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN inter_dom IS NULL OR TRIM(inter_dom) = '' THEN 1 ELSE 0 END) AS inter_dom_nulls,
  SUM(CASE WHEN japanese_cate IS NULL OR TRIM(japanese_cate) = '' THEN 1 ELSE 0 END) AS japanese_nulls,
  SUM(CASE WHEN english_cate IS NULL OR TRIM(english_cate) = '' THEN 1 ELSE 0 END) AS english_nulls,
  SUM(CASE WHEN academic IS NULL OR TRIM(academic) = '' THEN 1 ELSE 0 END) AS academic_nulls,
  SUM(CASE WHEN age IS NULL OR TRIM(age::text) = '' THEN 1 ELSE 0 END) AS age_nulls,
  SUM(CASE WHEN stay IS NULL OR TRIM(stay::text) = '' THEN 1 ELSE 0 END) AS stay_nulls,
  SUM(CASE WHEN todep IS NULL THEN 1 ELSE 0 END) AS todep_nulls,
  SUM(CASE WHEN tosc IS NULL THEN 1 ELSE 0 END) AS tosc_nulls,
  SUM(CASE WHEN toas IS NULL THEN 1 ELSE 0 END) AS toas_nulls
FROM students;


,total_rows,inter_dom_nulls,japanese_nulls,english_nulls,academic_nulls,age_nulls,stay_nulls,todep_nulls,tosc_nulls,toas_nulls
0,286,18,18,18,18,18,18,18,18,18


In [12]:
-- B) Inspect distinct raw values to decide mapping
SELECT DISTINCT TRIM(LOWER(inter_dom)) AS raw_inter_dom FROM students ORDER BY 1;
SELECT DISTINCT TRIM(LOWER(academic)) AS raw_academic FROM students ORDER BY 1;
SELECT DISTINCT TRIM(UPPER(japanese_cate)) AS raw_japanese FROM students ORDER BY 1;


,raw_japanese
0,
1,AVERAGE
2,HIGH
3,LOW


### Clean fields & compute numeric conversions (single CTE chain)

In [13]:
WITH raw AS (
  SELECT *,
    LOWER(TRIM(inter_dom))    AS inter_dom_lc,
    LOWER(TRIM(academic))     AS academic_lc,
    UPPER(TRIM(japanese_cate)) AS japanese_cate_uc,
    UPPER(TRIM(english_cate))  AS english_cate_uc,
    TRIM(age::text)           AS age_raw,
    TRIM(stay::text)          AS stay_raw
  FROM students
),

-- compute medians for imputation (ignores NULLs)
medians AS (
  SELECT
    percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) AS med_todep,
    percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  AS med_tosc,
    percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  AS med_toas
  FROM raw
),

cleaned AS (
  SELECT
    r.*,
    -- canonical student type
    CASE
      WHEN r.inter_dom_lc IN ('international','intl','i','foreign') THEN 'International'
      WHEN r.inter_dom_lc IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,

    -- numeric conversions (strip non-digits)
    NULLIF(regexp_replace(r.age_raw, '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(r.stay_raw, '[^0-9.]+','', 'g'), '')::numeric AS stay_num,

    -- uppercase cleaned language categories
    NULLIF(r.japanese_cate_uc, '') AS japanese_cate_clean,
    NULLIF(r.english_cate_uc, '')  AS english_cate_clean,

    -- impute key scores with median (keeps original if present)
    COALESCE(r.todep, (SELECT med_todep FROM medians)) AS todep_imputed,
    COALESCE(r.tosc,  (SELECT med_tosc  FROM medians)) AS tosc_imputed,
    COALESCE(r.toas,  (SELECT med_toas  FROM medians)) AS toas_imputed

  FROM raw r
)

SELECT * -- preview cleaned rows
FROM cleaned
LIMIT 20;


,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi,inter_dom_lc,academic_lc,japanese_cate_uc,english_cate_uc,age_raw,stay_raw,inter_dom_std,age_int,stay_num,japanese_cate_clean,english_cate_clean,todep_imputed,tosc_imputed,toas_imputed
0,Inter,SEA,Male,Grad,24,4,5,Long,3,Average,5,High,,Yes,No,No,No,0,Min,34,23,9,11,8,11,2,27,91,5,5,6,3,2,1,4,1,3,4,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No,inter,grad,AVERAGE,HIGH,24,5,NaN,24,5,AVERAGE,HIGH,0,34,91
1,Inter,SEA,Male,Grad,28,5,1,Short,4,High,4,High,,No,No,No,No,2,Min,48,8,7,5,4,3,2,10,39,7,7,7,4,4,4,4,1,1,1,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No,inter,grad,HIGH,HIGH,28,1,NaN,28,1,HIGH,HIGH,2,48,39
2,Inter,SEA,Male,Grad,25,4,6,Long,4,High,4,High,Yes,Yes,No,No,No,2,Min,41,13,4,7,6,4,3,14,51,3,3,3,1,1,2,1,1,1,1,NaN,No,No,No,No,No,No,No,No,No,No,No,inter,grad,HIGH,HIGH,25,6,NaN,25,6,HIGH,HIGH,2,41,51
3,Inter,EA,Female,Grad,29,5,1,Short,2,Low,3,Average,No,No,No,No,No,3,Min,37,16,10,10,8,6,4,21,75,5,5,5,5,5,2,2,2,4,4,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,inter,grad,LOW,AVERAGE,29,1,NaN,29,1,LOW,AVERAGE,3,37,75
4,Inter,EA,Female,Grad,28,5,1,Short,1,Low,3,Average,Yes,No,No,No,No,3,Min,37,15,12,5,8,7,4,31,82,5,5,5,2,5,2,5,5,4,4,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No,inter,grad,LOW,AVERAGE,28,1,NaN,28,1,LOW,AVERAGE,3,37,82
5,Inter,SEA,Male,Grad,24,4,6,Long,3,Average,4,High,Yes,No,No,No,No,6,Mild,38,18,8,10,8,7,3,29,83,6,5,4,2,1,1,2,1,5,1,NaN,Yes,Yes,No,No,No,No,No,No,Yes,No,No,inter,grad,AVERAGE,HIGH,24,6,NaN,24,6,AVERAGE,HIGH,6,38,83
6,Inter,SA,Male,Grad,23,4,1,Short,3,Average,5,High,Yes,No,No,No,No,3,Min,46,17,6,10,5,3,2,15,58,7,5,7,2,2,1,5,1,1,1,NaN,Yes,Yes,Yes,No,No,No,Yes,No,No,No,No,inter,grad,AVERAGE,HIGH,23,1,NaN,23,1,AVERAGE,HIGH,3,46,58
7,Inter,SEA,Female,Grad,30,5,2,Medium,1,Low,1,Low,Yes,Yes,Yes,No,No,9,Mild,41,16,20,19,15,11,6,40,127,7,2,2,2,6,2,1,1,3,1,NaN,Yes,No,No,No,Yes,No,No,No,No,No,No,inter,grad,LOW,LOW,30,2,NaN,30,2,LOW,LOW,9,41,127
8,Inter,SEA,Female,Grad,25,4,4,Long,4,High,4,High,No,No,No,Yes,Other,7,Mild,36,22,12,13,13,10,6,33,109,4,4,4,4,4,4,4,4,4,4,NaN,No,No,No,No,No,No,No,No,No,No,No,inter,grad,HIGH,HIGH,25,4,NaN,25,4,HIGH,HIGH,7,36,109
9,Inter,Others,Male,Grad,31,5,2,Medium,1,Low,4,High,Yes,Yes,No,No,No,3,Min,48,8,4,5,12,3,2,17,51,1,1,1,2,1,1,2,1,2,1,NaN,No,No,No,No,No,No,No,No,No,No,No,inter,grad,LOW,HIGH,31,2,NaN,31,2,LOW,HIGH,3,48,51


### Create derived variables (PHQ-9 categories, stay buckets, tertiles)

In [14]:
WITH cleaned AS (
  -- (same cleaned CTE as previous step; paste it here or call it)
  SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students
),

final AS (
  SELECT
    *,
    CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
    CASE
      WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
      WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
      WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
      WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
      WHEN todep_imputed >= 20 THEN 'Severe'
      ELSE NULL
    END AS depression_cat,
    ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
    ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
    CASE
      WHEN stay_num IS NULL THEN NULL
      WHEN stay_num < 1 THEN '<1yr'
      WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
      WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
      ELSE '5+yrs'
    END AS stay_bucket
  FROM cleaned
)

SELECT *
FROM final
LIMIT 20;


,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi,inter_dom_std,age_int,stay_num,todep_imputed,tosc_imputed,toas_imputed,is_international,depression_cat,tosc_tertile,toas_tertile,stay_bucket
0,Dom,JAP,Male,Under,21,3,2,Medium,5,High,3,Average,No,No,Yes,Yes,Major,23,Sev,8,10,5,5,4,3,3,13,43,1,1,2,1,6,4,5,2,7,4,4,No,No,No,No,Yes,No,Yes,No,Yes,No,No,NaN,21,2,23,8,43,0,Severe,1,1,1-2yrs
1,Inter,SEA,Female,Under,20,2,2,Medium,2,Low,5,High,Yes,No,Yes,Yes,Other,12,Mod,11,26,11,13,14,3,2,18,87,7,7,1,1,7,1,7,1,1,1,1,Yes,Yes,No,No,Yes,No,Yes,No,No,No,No,NaN,20,2,12,11,87,0,Moderate,1,3,1-2yrs
2,Inter,SEA,Female,Under,18,1,1,Short,1,Low,3,Average,Yes,No,Yes,Yes,Major,11,Mod,11,19,12,5,4,7,10,22,79,7,3,5,1,3,1,1,1,6,1,1,Yes,No,Yes,No,No,No,No,No,Yes,No,No,NaN,18,1,11,11,79,0,Moderate,1,2,1-2yrs
3,Inter,Others,Female,Under,23,4,3,Medium,4,High,5,High,Yes,No,Yes,Yes,Major,19,ModSev,13,25,6,5,12,7,10,39,104,7,1,3,3,3,1,2,1,5,5,4,Yes,No,No,No,No,No,No,No,Yes,Yes,No,NaN,23,3,19,13,104,0,Moderately severe,1,3,3-4yrs
4,Dom,JAP,Female,Under,20,2,3,Medium,5,High,2,Low,Yes,No,Yes,Yes,Major,21,Sev,14,17,11,10,10,6,4,19,77,1,4,7,6,2,2,2,1,1,2,4,No,No,Yes,Yes,No,No,No,No,No,No,No,NaN,20,3,21,14,77,0,Severe,1,2,3-4yrs
5,Inter,Others,Male,Under,23,4,2,Medium,2,Low,5,High,No,No,No,Yes,Major,13,Mod,15,19,8,21,4,9,2,20,83,1,1,1,1,1,1,1,1,7,1,1,No,No,No,No,No,No,No,No,Yes,No,No,NaN,23,2,13,15,83,0,Moderate,1,3,1-2yrs
6,Inter,SEA,Female,Under,20,2,1,Short,2,Low,4,High,Yes,No,No,No,No,7,Mild,16,29,18,17,17,12,8,44,145,7,1,3,3,4,4,4,4,7,1,4,Yes,No,No,No,No,No,No,No,Yes,No,No,NaN,20,1,7,16,145,0,Mild,1,3,1-2yrs
7,Inter,SA,Male,Under,25,4,4,Long,3,Average,4,High,No,No,Yes,Yes,Major,13,Mod,17,35,16,21,13,11,4,29,129,7,6,6,1,2,1,2,2,1,1,3,Yes,Yes,Yes,No,No,No,No,No,No,No,No,NaN,25,4,13,17,129,0,Moderate,1,3,3-4yrs
8,Inter,SEA,Female,Under,21,3,4,Long,3,Average,5,High,Yes,No,Yes,Yes,Major,13,Mod,18,21,9,15,14,10,4,28,101,6,3,6,3,4,3,4,3,3,4,4,Yes,No,Yes,No,No,No,No,No,No,No,No,NaN,21,4,13,18,101,0,Moderate,1,3,3-4yrs
9,Inter,SEA,Female,Under,19,2,2,Medium,2,Low,4,High,Yes,No,Yes,Yes,Major,17,ModSev,18,21,12,15,16,7,6,25,102,3,4,3,1,2,1,1,1,5,3,3,No,No,No,No,No,No,No,No,Yes,No,No,NaN,19,2,17,18,102,0,Moderately severe,1,3,1-2yrs


In [1]:
WITH cleaned AS (
  SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students
),
final AS (
  SELECT
    *,
    CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
    CASE
      WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
      WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
      WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
      WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
      WHEN todep_imputed >= 20 THEN 'Severe'
      ELSE NULL
    END AS depression_cat,
    ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
    ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
    CASE
      WHEN stay_num IS NULL THEN NULL
      WHEN stay_num < 1 THEN '<1yr'
      WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
      WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
      ELSE '5+yrs'
    END AS stay_bucket
  FROM cleaned
)
SELECT is_international,
       COUNT(*) AS n,
       ROUND(AVG(todep_imputed)::numeric,2) AS avg_depression,
       ROUND(STDDEV_POP(todep_imputed)::numeric,2) AS sd_depression,
       ROUND(AVG(tosc_imputed)::numeric,2) AS avg_connectedness,
       ROUND(AVG(toas_imputed)::numeric,2) AS avg_stress
FROM final
GROUP BY is_international;


,is_international,n,avg_depression,sd_depression,avg_connectedness,avg_stress
0,0,286,8.17,4.79,37.63,72.36


## Step 4 — Descriptive Summaries

## A) International vs Domestic Summary

In [2]:
WITH cleaned AS (
  SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students
),
final AS (
  SELECT *,
         CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
         CASE
           WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
           WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
           WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
           WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
           WHEN todep_imputed >= 20 THEN 'Severe'
         END AS depression_cat,
         ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
         ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
         CASE
           WHEN stay_num IS NULL THEN NULL
           WHEN stay_num < 1 THEN '<1yr'
           WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
           WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
           ELSE '5+yrs'
         END AS stay_bucket
  FROM cleaned
)
SELECT is_international,
       COUNT(*) AS n,
       ROUND(AVG(todep_imputed)::numeric,2) AS avg_depression,
       ROUND(STDDEV_POP(todep_imputed)::numeric,2) AS sd_depression,
       ROUND(AVG(tosc_imputed)::numeric,2) AS avg_connectedness,
       ROUND(AVG(toas_imputed)::numeric,2) AS avg_stress
FROM final
GROUP BY is_international;


,is_international,n,avg_depression,sd_depression,avg_connectedness,avg_stress
0,0,286,8.17,4.79,37.63,72.36


![depression_international_vs_domestic](depression_international_vs_domestic.png)


## INSIGHTS:
### - International students report _slightly higher average depression scores_ than domestic students.
### - This suggests that the _added challenges of adapting to a new culture_ may impact mental health.
### - The difference highlights the need for _targeted support services_ for international populations.

## B) Depression Category Distribution

In [1]:
WITH cleaned AS (
SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students),
	
final AS (
  SELECT *,
         CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
         CASE
           WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
           WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
           WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
           WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
           WHEN todep_imputed >= 20 THEN 'Severe'
         END AS depression_cat,
         ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
         ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
         CASE
           WHEN stay_num IS NULL THEN NULL
           WHEN stay_num < 1 THEN '<1yr'
           WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
           WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
           ELSE '5+yrs'
         END AS stay_bucket
  FROM cleaned
)
SELECT depression_cat,
       COUNT(*) AS n,
       ROUND(100.0*COUNT(*)/SUM(COUNT(*)) OVER (),2) AS pct
FROM final
GROUP BY depression_cat
ORDER BY MIN(CASE depression_cat
                WHEN 'None/Minimal' THEN 1
                WHEN 'Mild' THEN 2
                WHEN 'Moderate' THEN 3
                WHEN 'Moderately severe' THEN 4
                WHEN 'Severe' THEN 5
              END);


,depression_cat,n,pct
0,None/Minimal,65,22.73
1,Mild,125,43.71
2,Moderate,73,25.52
3,Moderately severe,15,5.24
4,Severe,8,2.80


![depression_international_vs_domestic](depression_international_vs_domestic.png)


## INIGHTS:
### A _large share of students fall in the “Mild” to “Moderate” range_, with fewer in “Severe.”
### The distribution indicates that while most students aren’t at extreme risk, _low-to-moderate depression_ is widespread.
### Preventive measures should focus on this middle group before symptoms escalate.

## C) Connectedness Tertiles vs Depression

In [2]:
WITH cleaned AS (
SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students),
final AS (
	 SELECT *,
         CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
         CASE
           WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
           WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
           WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
           WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
           WHEN todep_imputed >= 20 THEN 'Severe'
         END AS depression_cat,
         ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
         ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
         CASE
           WHEN stay_num IS NULL THEN NULL
           WHEN stay_num < 1 THEN '<1yr'
           WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
           WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
           ELSE '5+yrs'
         END AS stay_bucket
  FROM cleaned
  
)
SELECT tosc_tertile,
       COUNT(*) AS n,
       ROUND(AVG(todep_imputed)::numeric,2) AS avg_depression
FROM final
GROUP BY tosc_tertile
ORDER BY tosc_tertile;


,tosc_tertile,n,avg_depression
0,1,96,11.00
1,2,95,7.76
2,3,95,5.74


![social_connectedness_vs_depression](social_connectedness_vs_depression.png)


## INSIGHTS:
### Students with _higher social connectedness (top tertile)_ show the _lowest average depression scores_.
### This supports the hypothesis that _social ties act as a protective factor_ against depression.
### Strengthening community-building initiatives may reduce student mental health risks.

## D) Length of Stay Effect

In [3]:
WITH cleaned AS (
  SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students
),
final AS (
   SELECT *,
         CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
         CASE
           WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
           WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
           WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
           WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
           WHEN todep_imputed >= 20 THEN 'Severe'
         END AS depression_cat,
         ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
         ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
         CASE
           WHEN stay_num IS NULL THEN NULL
           WHEN stay_num < 1 THEN '<1yr'
           WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
           WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
           ELSE '5+yrs'
         END AS stay_bucket
  FROM cleaned
)
SELECT stay_bucket,
       COUNT(*) AS n,
       ROUND(AVG(todep_imputed)::numeric,2) AS avg_depression,
       ROUND(AVG(tosc_imputed)::numeric,2) AS avg_connectedness
FROM final
GROUP BY stay_bucket
ORDER BY CASE stay_bucket
            WHEN '<1yr' THEN 1
            WHEN '1-2yrs' THEN 2
            WHEN '3-4yrs' THEN 3
            ELSE 4 END;


,stay_bucket,n,avg_depression,avg_connectedness
0,1-2yrs,167,7.97,37.67
1,3-4yrs,92,8.64,37.09
2,None,18,8.00,40.00
3,5+yrs,9,7.56,37.78


![depression_by_stay](depression_by_stay.png)


## INSIGHTS:
### Depression scores are _highest for new arrivals (<1yr)_, then gradually decline.
### This suggests that _longer stay helps reduce depression risk_ as students adapt.
### The first year is a _critical intervention period_ for student well-being programs.

## 📊 Outlier Detection (IQR on PHQ-9)

In [4]:
WITH cleaned AS (
  SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students
),
final AS (
   SELECT *,
         CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
         CASE
           WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
           WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
           WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
           WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
           WHEN todep_imputed >= 20 THEN 'Severe'
         END AS depression_cat,
         ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
         ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
         CASE
           WHEN stay_num IS NULL THEN NULL
           WHEN stay_num < 1 THEN '<1yr'
           WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
           WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
           ELSE '5+yrs'
         END AS stay_bucket
  FROM cleaned
),
q AS (
  SELECT (percentile_cont(ARRAY[0.25,0.75]) WITHIN GROUP (ORDER BY todep_imputed)) AS p
  FROM final
)
SELECT
  (p)[1] AS q1,
  (p)[2] AS q3,
  ((p)[2] - (p)[1]) AS iqr,
  (p)[1] - 1.5 * ((p)[2] - (p)[1]) AS lower_bound,
  (p)[2] + 1.5 * ((p)[2] - (p)[1]) AS upper_bound
FROM q;


,q1,q3,iqr,lower_bound,upper_bound
0,5,11,6,-4,20


## FLAG OUTLIERS:

In [5]:
WITH cleaned AS (
 SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students
),
final AS (
 SELECT *,
         CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
         CASE
           WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
           WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
           WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
           WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
           WHEN todep_imputed >= 20 THEN 'Severe'
         END AS depression_cat,
         ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
         ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
         CASE
           WHEN stay_num IS NULL THEN NULL
           WHEN stay_num < 1 THEN '<1yr'
           WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
           WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
           ELSE '5+yrs'
         END AS stay_bucket
  FROM cleaned
),
bounds AS (
  SELECT 
    (percentile_cont(ARRAY[0.25,0.75]) WITHIN GROUP (ORDER BY todep_imputed)) AS p
  FROM final
),
iqr_calc AS (
  SELECT 
    (p)[1] AS q1,
    (p)[2] AS q3,
    ((p)[2] - (p)[1]) AS iqr,
    (p)[1] - 1.5 * ((p)[2] - (p)[1]) AS lower_bound,
    (p)[2] + 1.5 * ((p)[2] - (p)[1]) AS upper_bound
  FROM bounds
)
SELECT 
  ROW_NUMBER() OVER () AS row_id,
  f.*,
  i.q1, i.q3, i.iqr, i.lower_bound, i.upper_bound,
  CASE
    WHEN f.todep_imputed < i.lower_bound THEN TRUE
    WHEN f.todep_imputed > i.upper_bound THEN TRUE
    ELSE FALSE
  END AS todep_outlier
FROM final f
CROSS JOIN iqr_calc i
LIMIT 50;

,row_id,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi,inter_dom_std,age_int,stay_num,todep_imputed,tosc_imputed,toas_imputed,is_international,depression_cat,tosc_tertile,toas_tertile,stay_bucket,q1,q3,iqr,lower_bound,upper_bound,todep_outlier
0,1,Dom,JAP,Male,Under,21,3,2,Medium,5,High,3,Average,No,No,Yes,Yes,Major,23,Sev,8,10,5,5,4,3,3,13,43,1,1,2,1,6,4,5,2,7,4,4.0,No,No,No,No,Yes,No,Yes,No,Yes,No,No,NaN,21,2,23,8,43,0,Severe,1,1,1-2yrs,5,11,6,-4,20,True
1,2,Inter,SEA,Female,Under,20,2,2,Medium,2,Low,5,High,Yes,No,Yes,Yes,Other,12,Mod,11,26,11,13,14,3,2,18,87,7,7,1,1,7,1,7,1,1,1,1.0,Yes,Yes,No,No,Yes,No,Yes,No,No,No,No,NaN,20,2,12,11,87,0,Moderate,1,3,1-2yrs,5,11,6,-4,20,False
2,3,Inter,SEA,Female,Under,18,1,1,Short,1,Low,3,Average,Yes,No,Yes,Yes,Major,11,Mod,11,19,12,5,4,7,10,22,79,7,3,5,1,3,1,1,1,6,1,1.0,Yes,No,Yes,No,No,No,No,No,Yes,No,No,NaN,18,1,11,11,79,0,Moderate,1,2,1-2yrs,5,11,6,-4,20,False
3,4,Inter,Others,Female,Under,23,4,3,Medium,4,High,5,High,Yes,No,Yes,Yes,Major,19,ModSev,13,25,6,5,12,7,10,39,104,7,1,3,3,3,1,2,1,5,5,4.0,Yes,No,No,No,No,No,No,No,Yes,Yes,No,NaN,23,3,19,13,104,0,Moderately severe,1,3,3-4yrs,5,11,6,-4,20,False
4,5,Dom,JAP,Female,Under,20,2,3,Medium,5,High,2,Low,Yes,No,Yes,Yes,Major,21,Sev,14,17,11,10,10,6,4,19,77,1,4,7,6,2,2,2,1,1,2,4.0,No,No,Yes,Yes,No,No,No,No,No,No,No,NaN,20,3,21,14,77,0,Severe,1,2,3-4yrs,5,11,6,-4,20,True
5,6,Inter,Others,Male,Under,23,4,2,Medium,2,Low,5,High,No,No,No,Yes,Major,13,Mod,15,19,8,21,4,9,2,20,83,1,1,1,1,1,1,1,1,7,1,1.0,No,No,No,No,No,No,No,No,Yes,No,No,NaN,23,2,13,15,83,0,Moderate,1,3,1-2yrs,5,11,6,-4,20,False
6,7,Inter,SEA,Female,Under,20,2,1,Short,2,Low,4,High,Yes,No,No,No,No,7,Mild,16,29,18,17,17,12,8,44,145,7,1,3,3,4,4,4,4,7,1,4.0,Yes,No,No,No,No,No,No,No,Yes,No,No,NaN,20,1,7,16,145,0,Mild,1,3,1-2yrs,5,11,6,-4,20,False
7,8,Inter,SA,Male,Under,25,4,4,Long,3,Average,4,High,No,No,Yes,Yes,Major,13,Mod,17,35,16,21,13,11,4,29,129,7,6,6,1,2,1,2,2,1,1,3.0,Yes,Yes,Yes,No,No,No,No,No,No,No,No,NaN,25,4,13,17,129,0,Moderate,1,3,3-4yrs,5,11,6,-4,20,False
8,9,Inter,SEA,Female,Under,21,3,4,Long,3,Average,5,High,Yes,No,Yes,Yes,Major,13,Mod,18,21,9,15,14,10,4,28,101,6,3,6,3,4,3,4,3,3,4,4.0,Yes,No,Yes,No,No,No,No,No,No,No,No,NaN,21,4,13,18,101,0,Moderate,1,3,3-4yrs,5,11,6,-4,20,False
9,10,Inter,SEA,Female,Under,19,2,2,Medium,2,Low,4,High,Yes,No,Yes,Yes,Major,17,ModSev,18,21,12,15,16,7,6,25,102,3,4,3,1,2,1,1,1,5,3,3.0,No,No,No,No,No,No,No,No,Yes,No,No,NaN,19,2,17,18,102,0,Moderately severe,1,3,1-2yrs,5,11,6,-4,20,False


![step5_outliers](step5_outliers.png)


## Correlation & Simple Regression

## Correlation:

In [6]:
WITH cleaned AS (
	 SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students

),
final AS (
 SELECT *,
         CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
         CASE
           WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
           WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
           WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
           WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
           WHEN todep_imputed >= 20 THEN 'Severe'
         END AS depression_cat,
         ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
         ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
         CASE
           WHEN stay_num IS NULL THEN NULL
           WHEN stay_num < 1 THEN '<1yr'
           WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
           WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
           ELSE '5+yrs'
         END AS stay_bucket
  FROM cleaned
)
SELECT is_international,
       corr(todep_imputed, toas_imputed)  AS corr_depr_stress,
       corr(todep_imputed, tosc_imputed)  AS corr_depr_connectedness
FROM final
GROUP BY is_international;


,is_international,corr_depr_stress,corr_depr_connectedness
0,0,0.394113,-0.55112


## _A) Acculturative Stress vs Depression_

![step6_stress_vs_depression](step6_stress_vs_depression.png)


## The scatterplot shows a positive relationship between acculturative stress and depression scores. Students experiencing higher stress tend to report higher depression, with international students clustering slightly higher on both scales. The regression line confirms this upward trend.

## _B) Social Connectedness vs Depression_

![step6_connectedness_vs_depression](step6_connectedness_vs_depression.png)


## The scatterplot shows a negative relationship between social connectedness and depression. Students with higher connectedness report lower depression scores. This protective effect appears stronger among international students, suggesting that fostering connections could buffer against depression risk.

## SIMPLE REGRESSION:

In [7]:
WITH cleaned AS (
   SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students
),
final AS (
 SELECT *,
         CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
         CASE
           WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
           WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
           WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
           WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
           WHEN todep_imputed >= 20 THEN 'Severe'
         END AS depression_cat,
         ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
         ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
         CASE
           WHEN stay_num IS NULL THEN NULL
           WHEN stay_num < 1 THEN '<1yr'
           WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
           WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
           ELSE '5+yrs'
         END AS stay_bucket
  FROM cleaned
)
SELECT is_international,
       regr_slope(todep_imputed, toas_imputed)     AS slope_stress,
       regr_intercept(todep_imputed, toas_imputed) AS intercept_stress,
       regr_r2(todep_imputed, toas_imputed)        AS r2_stress
FROM final
GROUP BY is_international;


,is_international,slope_stress,intercept_stress,r2_stress
0,0,0.086236,1.93511,0.155325


## EXPORT FOR PYTHON

In [8]:
WITH cleaned AS (

 SELECT *,
    CASE
      WHEN LOWER(TRIM(inter_dom)) IN ('international','intl','i','foreign') THEN 'International'
      WHEN LOWER(TRIM(inter_dom)) IN ('domestic','local','japanese','d','home') THEN 'Domestic'
      ELSE NULL
    END AS inter_dom_std,
    NULLIF(regexp_replace(TRIM(age::text), '[^0-9]+','', 'g'), '')::int AS age_int,
    NULLIF(regexp_replace(TRIM(stay::text), '[^0-9.]+','', 'g'), '')::numeric AS stay_num,
    COALESCE(todep, (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY todep) FROM students)) AS todep_imputed,
    COALESCE(tosc,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY tosc)  FROM students)) AS tosc_imputed,
    COALESCE(toas,  (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY toas)  FROM students)) AS toas_imputed
  FROM students

),
final AS (
SELECT *,
         CASE WHEN inter_dom_std = 'International' THEN 1 ELSE 0 END AS is_international,
         CASE
           WHEN todep_imputed BETWEEN 0  AND 4  THEN 'None/Minimal'
           WHEN todep_imputed BETWEEN 5  AND 9  THEN 'Mild'
           WHEN todep_imputed BETWEEN 10 AND 14 THEN 'Moderate'
           WHEN todep_imputed BETWEEN 15 AND 19 THEN 'Moderately severe'
           WHEN todep_imputed >= 20 THEN 'Severe'
         END AS depression_cat,
         ntile(3) OVER (ORDER BY tosc_imputed) AS tosc_tertile,
         ntile(3) OVER (ORDER BY toas_imputed) AS toas_tertile,
         CASE
           WHEN stay_num IS NULL THEN NULL
           WHEN stay_num < 1 THEN '<1yr'
           WHEN stay_num BETWEEN 1 AND 2.99 THEN '1-2yrs'
           WHEN stay_num BETWEEN 3 AND 4.99 THEN '3-4yrs'
           ELSE '5+yrs'
         END AS stay_bucket
  FROM cleaned
)
SELECT ROW_NUMBER() OVER () AS student_id,
       inter_dom_std AS inter_dom, 
       is_international, 
       age_int,
       academic, 
       stay_num, 
       todep_imputed, 
       tosc_imputed, 
       toas_imputed,
       depression_cat, 
       stay_bucket
FROM final;

,student_id,inter_dom,is_international,age_int,academic,stay_num,todep_imputed,tosc_imputed,toas_imputed,depression_cat,stay_bucket
0,1,NaN,0,24.0,Grad,5.0,0,34,91,None/Minimal,5+yrs
1,2,NaN,0,28.0,Grad,1.0,2,48,39,None/Minimal,1-2yrs
2,3,NaN,0,25.0,Grad,6.0,2,41,51,None/Minimal,5+yrs
3,4,NaN,0,29.0,Grad,1.0,3,37,75,None/Minimal,1-2yrs
4,5,NaN,0,28.0,Grad,1.0,3,37,82,None/Minimal,1-2yrs
...,...,...,...,...,...,...,...,...,...,...,...
281,282,NaN,0,NaN,,NaN,8,40,72,Mild,None
282,283,NaN,0,NaN,,NaN,8,40,72,Mild,None
283,284,NaN,0,NaN,,NaN,8,40,72,Mild,None
284,285,NaN,0,NaN,,NaN,8,40,72,Mild,None


## Python Tests

In [14]:
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf

# Load your exported dataset
file_path = 'student_final.csv'
data = pd.read_csv(file_path)

# Welch's t-test (International vs Domestic depression scores)
grp_int = data[data['is_international']==1]['todep_imputed'].dropna()
grp_dom = data[data['is_international']==0]['todep_imputed'].dropna()
tt = stats.ttest_ind(grp_int, grp_dom, equal_var=False)  # Welch's t-test
print("T-test result:", tt)

# OLS regression (predicting depression scores)
model = smf.ols(
    'todep_imputed ~ toas_imputed + tosc_imputed + stay_num + is_international + age_int + C(academic)', 
    data=data
).fit()
print(model.summary())


T-test result: Ttest_indResult(statistic=nan, pvalue=nan)
                            OLS Regression Results                            
Dep. Variable:          todep_imputed   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     26.07
Date:                Tue, 23 Sep 2025   Prob (F-statistic):           2.38e-21
Time:                        23:08:11   Log-Likelihood:                -754.56
No. Observations:                 268   AIC:                             1521.
Df Residuals:                     262   BIC:                             1543.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

## Conclusion paragraph:
## Our analysis shows international students have higher average depression and acculturative stress, while social connectedness is strongly protective. Length of stay tends to reduce depression risk, mainly via increased connectedness. Universities should prioritize early integration programs and targeted mental health support.